In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from glob import glob
from progiter import ProgIter
import joblib

from sklearn import metrics

import torch
from torch.utils.data import TensorDataset, DataLoader

from transformers import AutoTokenizer

import warnings
warnings.filterwarnings('ignore')

# 1. Функции

In [2]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Расчет метрики Hamming score, отражающей долю верно предсказанных
    элементов в объекте для задачи multi-label классификации
    '''
    acc_list = []
    
    # Цикл проходится по каждой паре таргет-предикт
    for i in range(y_true.shape[0]):
        # Для таргета и предикта создаются множества из индеков, на которых стоят единички
        set_true = set(np.where(y_true[i])[0])
        set_pred = set(np.where(y_pred[i])[0])
        tmp_a = None
        
        # Если оба вектора состоят толька из ноликов, то предсказание абсолютно верно, и его скор = 1
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        
        # В противном случае количество верно предсказанных единичек делится на количество элементов
        # в объединенном множесте индексов единичек таргета и предикта. Таким образом, скор будет = 1
        # только в случае, если верно найдены все единички и нет ложных единичек
        else:
            tmp_a = len(set_true.intersection(set_pred)) / float(len(set_true.union(set_pred)))

        acc_list.append(tmp_a)
    return np.mean(acc_list)


def evaluate(y_true, y_pred):
    metrics_dict = {
        'accuracy': metrics.accuracy_score(y_true, y_pred),
        'hamming': hamming_score(y_true, y_pred),
        'f1_score_micro': metrics.f1_score(y_true, y_pred, average='micro'),
        'f1_score_macro': metrics.f1_score(y_true, y_pred, average='macro'),
        'recall_score_micro': metrics.recall_score(y_true, y_pred, average='micro'),
        'recall_score_macro': metrics.recall_score(y_true, y_pred, average='macro'),
        'precision_score_micro': metrics.precision_score(y_true, y_pred, average='micro',
                                                         zero_division=0.0),
        'precision_score_macro': metrics.precision_score(y_true, y_pred, average='macro',
                                                         zero_division=0.0)
    }
    return metrics_dict

In [3]:
def make_dataset(texts, labels):
    '''
    Токенизация текстов и сопоставление токенов с идентификаторами
    соответствующих им слов. Формирование PyTorch датасета
    '''
    input_ids = []        # Список для токенизированных текстов
    attention_masks = []  # Список для масок механизма внимания
    
    # Цикл проходится и токенизирует каждый текст
    for sent in texts:
        encoded_dict = tokenizer.encode_plus(
            sent,                        # Последовательность для токенизации
            add_special_tokens=True,     # Добавить специальные токены в начало и в конец посл-ти
            max_length=338,              # Максимальная длина последовательности
            padding='max_length',        # Токен для заполнения до максимальной длины
            return_attention_mask=True,  # Маска механизма внимания для указания на паддинги
            return_tensors = 'pt',       # Возвращать pytorch-тензоры
            truncation=True              # Обрезать последовательность до максимальной длины
        )

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    
    # Конкатенация входных данных в тензоры
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    # Преобразование таргетов в тензоры
    labels = torch.tensor(labels.values)
    # Формирование датасета
    dataset = TensorDataset(input_ids, attention_masks, labels)

    return dataset

In [4]:
def validate():
    '''
    Валидация обученной модели на тестовой выборке
    '''
    print(f'Validation')
    model.eval()             # Перевод модели в режим валидации
    fin_targets = []         # Список для всех таргетов валидационной выборки
    fin_outputs = []         # Список для всех предиктов модели на валидационной выборки
    total_test_loss = 0.0    # Лосс на валидации
    
    with torch.no_grad():
        # Без подсчета градиентов цикл проходится по батчам, Progiter отображает шкалу прогресса
        for data in ProgIter(test_dataloader):
            ids = data[0].to(device, dtype=torch.long)            # Токены последовательностей из батча
            mask = data[1].to(device, dtype=torch.long)           # Маски механизма внимания посл-тей
            targets = data[2].to(device, dtype=torch.float)       # Таргеты из батча
                
            res = model(ids, attention_mask=mask, labels=targets) # В модель подаются входные тензоры и таргеты
            loss = res['loss']                                    # Вычисляется значение функции потерь
            logits = res['logits']                                # Логиты предсказаний модели
            total_test_loss += loss.item()                        # Складывается лосс
            
            # Таргеты и выходы модели по батчу добавляются в списки. Логиты проходят через сигмоиду
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(logits).cpu().detach().numpy().tolist())
    
    fin_targets = np.array(fin_targets)
    fin_outputs = np.array(fin_outputs)
    predictions = np.zeros(fin_outputs.shape)
    predictions[np.where(fin_outputs >= 0.5)] = 1
    
    return total_test_loss / len(test_dataloader), fin_targets, predictions

In [5]:
def fill_gpt_df(data, col_name):
    for row, pred in enumerate(data.astype(int)):
        genres = [genre for genre, match in zip(y_gpt.columns, pred) if match]
        gpt_df.loc[row, col_name] = ', '.join(genres)

# 2. Датасет

In [6]:
df_test = pd.read_csv('kinopoisk_test.csv')
X_test, y_test = df_test['descr_lemmas'], df_test.drop('descr_lemmas', axis=1)

df_gpt = pd.read_csv('gpt_test.csv')
X_gpt, y_gpt = df_gpt['descr_lemmas'], df_gpt.drop('descr_lemmas', axis=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# 3. Создание сводных таблиц

In [7]:
metrics_df = pd.DataFrame()
gpt_df = pd.DataFrame({'descr_lemmas': X_gpt})
fill_gpt_df(y_gpt.values, 'true_genres')
models = []

for filename in glob('models/*', recursive=True):
    models.append(filename)

for model_name in models:
    if '.pkl' in model_name:
        row_no = len(metrics_df)
        metrics_df.loc[row_no, 'model_name'] = model_name[7:-4]
        model = joblib.load(model_name)
        y_pred = model.predict(X_test)
        metrics_dict = evaluate(y_test.to_numpy(), y_pred)
        for metric, value in metrics_dict.items():
            metrics_df.loc[row_no, metric] = value
        y_pred = model.predict(X_gpt)
    else:
        row_no = len(metrics_df)
        model_name = re.sub(r'(\w{3})(-)', r'\1/', model_name[7:-3], 1)
        metrics_df.loc[row_no, 'model_name'] = model_name
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        test_dataset = make_dataset(X_test, y_test)
        test_dataloader = DataLoader(test_dataset)
        model = torch.load(f"models/{model_name.replace('/', '-')}.pt")
        avg_val_loss, targets, y_pred = validate()
        metrics_dict = evaluate(targets, y_pred)
        for metric, value in metrics_dict.items():
            metrics_df.loc[row_no, metric] = value
        metrics_df.loc[row_no, 'avg_val_loss'] = avg_val_loss
        test_dataset = make_dataset(X_gpt, y_gpt)
        test_dataloader = DataLoader(test_dataset)
        avg_val_loss, targets, y_pred = validate()
    fill_gpt_df(y_pred, f'{model_name}')

Validation
 100.00% 21536/21536... rate=36.25 Hz, eta=0:00:00, total=0:11:36
Validation
 100.00% 10/10... rate=36.42 Hz, eta=0:00:00, total=0:00:00
Validation
 100.00% 21536/21536... rate=252.41 Hz, eta=0:00:00, total=0:01:59
Validation
 100.00% 10/10... rate=186.15 Hz, eta=0:00:00, total=0:00:00
Validation
 100.00% 21536/21536... rate=36.85 Hz, eta=0:00:00, total=0:09:57
Validation
 100.00% 10/10... rate=36.13 Hz, eta=0:00:00, total=0:00:00
Validation
 100.00% 21536/21536... rate=71.13 Hz, eta=0:00:00, total=0:05:29
Validation
 100.00% 10/10... rate=71.30 Hz, eta=0:00:00, total=0:00:00


# 4. Сводные таблицы

In [8]:
metrics_df

,model_name,accuracy,hamming,f1_score_micro,f1_score_macro,recall_score_micro,recall_score_macro,precision_score_micro,precision_score_macro,avg_val_loss
0,ai-forever/rubert-base,0.207327,0.496966,0.602233,0.496592,0.564724,0.454800,0.645079,0.601281,0.159066
1,cointegrated/rubert-tiny2,0.184807,0.456611,0.562067,0.430205,0.504852,0.375983,0.633908,0.544965,0.166196
2,facebookai/xlm-roberta-base,0.198644,0.485977,0.591617,0.491563,0.548888,0.441998,0.641562,0.584751,0.165794
3,geotrend/distilbert-base-ru-cased,0.192376,0.471496,0.579495,0.477223,0.531742,0.421753,0.636673,0.574781,0.168679
4,tfidf_catboost_ngram_1_1,0.130340,0.326179,0.448542,0.335082,0.338136,0.244361,0.666002,0.603070,NaN
5,tfidf_catboost_ngram_1_2,0.130386,0.352952,0.481848,0.369556,0.390209,0.290363,0.629740,0.562383,NaN
6,tfidf_logreg_ngram_1_1,0.073366,0.415374,0.554078,0.478338,0.699192,0.644818,0.458847,0.384941,NaN
7,tfidf_logreg_ngram_1_2,0.102944,0.442976,0.576969,0.493959,0.683436,0.601088,0.499203,0.427817,NaN


In [9]:
gpt_df

,descr_lemmas,true_genres,ai-forever/rubert-base,cointegrated/rubert-tiny2,facebookai/xlm-roberta-base,geotrend/distilbert-base-ru-cased,models\tfidf_catboost_ngram_1_1.pkl,models\tfidf_catboost_ngram_1_2.pkl,models\tfidf_logreg_ngram_1_1.pkl,models\tfidf_logreg_ngram_1_2.pkl
0,далекий будущее человечество распространяться ...,"боевик, приключения, фантастика","боевик, приключения, фантастика","боевик, мультфильм, приключения, фантастика","боевик, приключения, фантастика","боевик, приключения, фантастика","боевик, мультфильм, приключения, фантастика","боевик, мультфильм, приключения, фантастика, ф...","аниме, боевик, мультфильм, приключения, семейн...","аниме, боевик, мультфильм, приключения, семейн..."
1,тихий прибрежный городок лэйквью жизнь идти св...,"детектив, драма, триллер","детектив, драма, криминал, триллер","детектив, драма, криминал, триллер","детектив, драма, криминал, триллер","детектив, драма, криминал, триллер","детектив, драма","детектив, драма, криминал, триллер","детектив, драма, криминал, триллер","детектив, драма, криминал, триллер"
2,маленький дружный городок мячград весь помешан...,"комедия, мультфильм, спорт","комедия, мультфильм, семейный, спорт","комедия, мультфильм, семейный","комедия, мультфильм, семейный","детский, комедия, мультфильм, семейный","комедия, спорт","комедия, мультфильм, семейный, спорт","детский, комедия, мультфильм, приключения, сем...","детский, комедия, мультфильм, приключения, сем..."
3,сквозь пламень это эпический история любовь му...,"биография, военный, мелодрама","военный, драма, мелодрама","военный, драма, история, мелодрама","военный, драма, мелодрама","военный, драма, мелодрама",драма,"военный, драма, музыка","биография, военный, драма, история, мелодрама,...","биография, военный, драма, история, мелодрама,..."
4,лос-анджелес год город ангел скрывать свой нео...,фильм-нуар,"детектив, драма, криминал, триллер","драма, криминал, триллер","детектив, драма, криминал, триллер","драма, криминал, триллер",драма,"детектив, драма, криминал, триллер","боевик, детектив, драма, криминал, триллер","боевик, детектив, драма, криминал, триллер"
5,великобритания конец век небольшой городок хар...,"история, мюзикл, семейный","драма, мелодрама, музыка, мюзикл","драма, музыка","драма, мелодрама, музыка","драма, мелодрама","драма, музыка","драма, комедия, музыка","драма, мелодрама, музыка, мюзикл, семейный","драма, мелодрама, музыка, мюзикл, семейный"
6,маленький японский городок окумар окружать гус...,"аниме, ужасы",ужасы,ужасы,ужасы,ужасы,"аниме, ужасы, фэнтези","триллер, ужасы, фэнтези","аниме, детектив, триллер, ужасы, фэнтези","аниме, детектив, триллер, ужасы, фантастика, ф..."
7,забывать герой дикий запад это короткометражны...,"вестерн, документальный, короткометражка",вестерн,документальный,вестерн,вестерн,"вестерн, документальный","биография, вестерн, документальный","биография, вестерн, документальный, история","биография, вестерн, документальный, история"
8,маленький деревня край густой лес жить десятил...,"детский, фэнтези","приключения, семейный, фэнтези","мультфильм, приключения, семейный, фэнтези","семейный, фэнтези","приключения, семейный, фэнтези","мультфильм, фэнтези","мультфильм, приключения, фэнтези","детский, мультфильм, мюзикл, приключения, семе...","детский, мультфильм, приключения, семейный, фэ..."
9,тайна правосудие захватывать реальный тв шоу с...,"криминал, реальное ТВ, ток-шоу",документальный,,детектив,детектив,криминал,"детектив, ток-шоу","детектив, документальный, криминал, реальное Т...","детектив, документальный, криминал, реальное Т..."


In [10]:
metrics_df.to_csv('metrics.csv', index=False)
gpt_df.to_csv('gpt_preds.csv', index=False)